In [0]:
%pip install torch transformers faiss-cpu numpy rouge-score nltk sacrebleu
%pip install sentence-transformers
%pip install langchain
%restart_python


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# import libraries
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import faiss
import time
import psutil
from functools import lru_cache
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

2024-09-25 16:25:09.179252: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 16:25:09.184293: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-25 16:25:09.252366: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 16:25:10.978804: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/local_disk0/.ephemeral_nfs/envs/pythonEnv-eca87918-1325-4aaf-bd3c-9a67f9511ac6/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be abl

In [0]:
# define class fit the format 
class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata or {}

# Load definition files
def load_definitions(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file if line.strip()]

In [0]:
# Generate embeddings for definitions
class Retriever:
    def __init__(self, definitions, num_retrieved_docs=5):
        all_documents = [Document(definition) for definition in definitions]
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        self.db = FAISS.from_documents(all_documents, embeddings)
        self.retriever = self.db.as_retriever(search_kwargs={"k": num_retrieved_docs})
        self.index = self.db.index

    def search(self, query):
        return self.retriever.get_relevant_documents(query)

In [0]:
# setup the generation model
class T5Assistant:
    def __init__(self, model_name='t5-small'):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def create_prompt(self, query, retrieved_info):
        return (f"Explain the concept or answer the question in a detailed manner using simple words and examples.\n"
                f"Instruction: {query}\n"
                f"Relevant information: {retrieved_info}\n"
                f"Output:")

    def generate_reply(self, query, retrieved_info):
        prompt = self.create_prompt(query, retrieved_info)
        input_ids = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).input_ids
        outputs = self.model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [0]:
def calculate_bleu(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    smoothie = SmoothingFunction().method4 
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)
  
def print_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    print(f"Memory Usage: {memory_info.rss / 1024 / 1024:.2f} MB")

@lru_cache(maxsize=10)
def cached_generate_reply(assistant, query, retrieved_info):
    return assistant.generate_reply(query, retrieved_info)

In [0]:
if __name__ == "__main__":
    # load the data
    definitions = load_definitions('ctx_pd.txt')
    
    # initialized the models
    retriever = Retriever(definitions, num_retrieved_docs=5)
    assistant = T5Assistant(model_name='t5-small')

    # create sample data
    generated_queries = [
        "What is accreditation?",
        "How is assessment done?",
        "What is a bachelor's degree?",
        "What is a campus?"
    ]

    # calculate the bleu score
    for query in generated_queries:
        # recore time
        start_time = time.time()
        retrieved_docs = retriever.search(query)
        retrieved_info = " ".join([doc.page_content for doc in retrieved_docs])

        # find the referance answer in the rag
        reference_answer = retrieved_docs[0].page_content if retrieved_docs else ""

        generated_reply = cached_generate_reply(assistant, query, retrieved_info)

        if reference_answer:
            bleu_score = calculate_bleu(reference_answer, generated_reply)
        else:
            bleu_score = "N/A"  

        # print out the solution
        print(f"Query: {query}")
        print(f"Generated Reply:\n{generated_reply}")
        print(f"Reference Answer:\n{reference_answer}")
        print(f"BLEU Score: {bleu_score}")

        print_memory_usage()

        end_time = time.time()
        print(f"Time taken for this query: {end_time - start_time:.2f} seconds\n")

/root/.ipykernel/4900/command-756413098566022-602137293:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been add

Query: What is accreditation?
Generated Reply:
accreditation? Relevant information: Academic Accreditation is Approval process by an accrediting authority of a learning program leading to an AQF qualification. OR the evaluation of a course of study (either by TEQSA or by a self-accrediting provider such as a university) against course requirements specified in the Higher Education Standards Framework.
Reference Answer:
Academic Accreditation is Approval process by an accrediting authority of a learning program leading to an AQF qualification using the quality assurance standards for the relevant education and training sector OR the evaluation of a course of study (either by TEQSA or by a self-accrediting provider such as a university) against course requirements specified in the Higher Education Standards Framework. created by AQF (Australian Qualifications Framework)
BLEU Score: 0.6617022713434835
Memory Usage: 1582.30 MB
Time taken for this query: 4.53 seconds

Query: How is assessme